In [8]:
import numpy as np
import random

In [71]:
def ABCindeks(drevo): #izracun ABC indeksa za neko drevo, len(drevo[u]) ti da dolzino seznama sosedov za vozlisce u torej je to stopnja vozlisca u
    produkt = 1
    for u in drevo:
        for v in drevo[u]:
            produkt = produkt * (len(drevo[u]) + len(drevo[v]) - 2) / (len(drevo[u]) * len(drevo[v]))
    return sqrt(produkt)

#ABCindeks(G)

In [2]:
def minABCindeks(n): #izracun najmanjsega ABC indeksa za vsa drevesa z n vozlisci, vrne tudi seznam dreves, ki imajo ta najmanjsi indeks
    min_indeks = None
    for dr in graphs.trees(n):
        if min_indeks == None:
            min_indeks = ABCindeks(dr)
            drevesa_z_min_ind = [dr]
        else:
            if ABCindeks(dr) < min_indeks:
                min_indeks = ABCindeks(dr)
                drevesa_z_min_ind = [dr]
            elif ABCindeks(dr) == min_indeks:
                drevesa_z_min_ind.append(dr)
            else:
                continue
    return min_indeks , drevesa_z_min_ind

#minABCindeks(9)

In [4]:
rezultati = []
for n in range(17) : #range naredi seznam od 0 do 18, midva hoceva grafe z vozlisci od 1 do 19 zato je spodaj n+1
        rez = [vrednost, seznam_grafov] = minABCindeks(n+1)
        rezultati.append(rez)
rezultati #rabu je okol 10 min (za 19)

[(1, [Graph on 1 vertex]),
 (0, [Graph on 2 vertices]),
 (1/4, [Graph on 3 vertices]),
 (1/8, [Graph on 4 vertices]),
 (1/16, [Graph on 5 vertices]),
 (1/32, [Graph on 6 vertices]),
 (1/64, [Graph on 7 vertices, Graph on 7 vertices]),
 (1/128, [Graph on 8 vertices, Graph on 8 vertices]),
 (1/256,
  [Graph on 9 vertices,
   Graph on 9 vertices,
   Graph on 9 vertices,
   Graph on 9 vertices]),
 (1/576, [Graph on 10 vertices]),
 (1/1152, [Graph on 11 vertices]),
 (5/12288, [Graph on 12 vertices]),
 (1/5184, [Graph on 13 vertices]),
 (3/32768, [Graph on 14 vertices]),
 (25/589824, [Graph on 15 vertices]),
 (25/1179648, [Graph on 16 vertices, Graph on 16 vertices]),
 (5/524288, [Graph on 17 vertices])]

In [172]:
round((1/31104)/(25/1179648),5)

1.51704

In [9]:
# recimo da naju zanimajo premeri teh grafov, uporabiva lahko metodo diameter za objekte graphs
premeri = []
for (vrednost, seznam_grafov) in rezultati:
    pr = []
    for graf in seznam_grafov:
        pr.append(graf.diameter())
    premeri.append(pr)
premeri

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6, 4],
 [7, 5],
 [8, 6, 6, 4],
 [5],
 [6],
 [5],
 [6],
 [5],
 [6],
 [7, 6],
 [6],
 [6],
 [6]]

In [11]:
#ce naju zanimajo dolzine najdaljsih poti v grafih 
def najdaljsa_pot(graf):
    najdaljsa = 0
    for u in graf:            #gremo cez vsa vozlisca v grafu
        if len(graf[u]) == 2:  #ce imajo stopnjo = 2 nadaljujemo
            soseda = graf[u]
            s = u #dolocimo sedanje vozlisce
            dolzina = 3        #sedaj imamo pot dolzine vsaj 3 - sedanje vozlišče in njegova soseda
            if len(graf[soseda[0]]) != 2: #preverimo ali ima vsaj eno od sosednjih vozlisc stopnjo neenako 2 (da ne zacnemo steti sredi poti)
                p = soseda[0] #dolocimo prejsnje vozlisce (tisto s stopnjo neenako 2)
                n = soseda[1] # in naslednje vozlisce
            elif len(graf[soseda[1]]) != 2:
                p = soseda[1]
                n = soseda[0]
            else: #ce imata obe sosednji vozlisci stopnjo 2 smo na sredi poti in tega primera ne preverjamo
                continue
            while len(graf[n]) == 2: #z while zanko se bomo sedaj premikali po poti dokler ne naletimo na vozlisce ki nima stopnje = 2
                dolzina += 1 #za vsako naslednje vozlisce povecamo dolzino
                p = s #premanemo vozlisca: novo prejsnje je staro sedanje,
                s = n #novo sedanje je staro naslednje
                if graf[s][0] != p: #novo naslednje je tisto od sosedov novega sedanjega ki ni enako novemu prejsnjemu
                    n = graf[s][0]
                else:
                    n = graf[s][1]
        else:
            continue
        if dolzina > najdaljsa: #ce smo nasli novo najdaljso pot jo shranimo
            najdaljsa = dolzina
    return najdaljsa

najdaljse_poti = []
for (vrednost, seznam_grafov) in rezultati:
    dol = []
    for graf in seznam_grafov:
        dol.append(najdaljsa_pot(graf))
    najdaljse_poti.append(dol)

najdaljse_poti

[[0],
 [0],
 [3],
 [4],
 [5],
 [6],
 [7, 3],
 [8, 4],
 [9, 5, 4, 3],
 [3],
 [4],
 [3],
 [3],
 [3],
 [3],
 [4, 4],
 [3],
 [3],
 [3]]

In [163]:
def SA_algoritem(n, T=100, fun=ABCindeks):
    drevo =graphs.RandomTree(n)
    scale = np.sqrt(T)
    trenutna_vrednost = fun(drevo)
    history = [drevo]
    for i in range(10000):
        listi_drevesa = []
        stopnje_vozlisc = drevo.degree()
        for i in range(n):
            if stopnje_vozlisc[i] == 1:
                listi_drevesa.append(i)               ### sedaj imamo vse liste drevesa
        stevilo_listov = len(listi_drevesa)
        vozlisca_ki_niso_listi = [x for x in drevo.vertices() if x not in listi_drevesa]
        nakljucni_list = listi_drevesa[random.randint(0,stevilo_listov - 1)] ### izberemo nakljucni list
        povezava = (nakljucni_list, drevo[nakljucni_list][0]) ### list bo imel le enega soseda
        popravek = copy(drevo)   ### ustvarimo kopijo drevesa na katerem bomo izvedli zamenjavo
        popravek.delete_edge(povezava)
        izbrano_vozlisce = vozlisca_ki_niso_listi[random.randint(0, len(vozlisca_ki_niso_listi) -1)]
        popravek.add_edge(nakljucni_list, izbrano_vozlisce) #### dodamo povezavo na kopiji
        vrednost_popravka = fun(popravek)
        if (trenutna_vrednost < vrednost_popravka) or (np.exp(-(trenutna_vrednost - vrednost_popravka)/T)): ### popravek je zavrzen ce je bila vrednost popravka vecja od trenutne vrednosti
            popravek = drevo
        drevo = popravek
        trenutna_vrednost = fun(drevo)
        T = 0.9*T
        history.append(drevo)
    return trenutna_vrednost, fun(history[0])

In [173]:
SA_algoritem(16)

In [171]:
1/31104 < 1/27648

True